In [2]:
from keys import OPENAI_API_KEY
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [3]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.6)
user = {
    'name': 'Charles',
    'age': 21,
    'sex': 'male'
}

In [4]:
from gcal import get_day_events
from weather import fetch_weather_forecast

events = get_day_events()
forecast_data = fetch_weather_forecast(
    latitude=37.8716,
    longitude=-122.2728,
    hourly="temperature_2m,precipitation_probability,precipitation",
    daily="temperature_2m_max,temperature_2m_min,sunrise,sunset",
    temperature_unit="fahrenheit",
    timezone="America/Los_Angeles",
    forecast_days=1
)

Getting the events for 2023-08-28T00:00:00-07:00
[{'kind': 'calendar#event', 'etag': '"3386363828252000"', 'id': '7je0lph6k52uknelid0an4h8lf_20230828T143000Z', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=N2plMGxwaDZrNTJ1a25lbGlkMGFuNGg4bGZfMjAyMzA4MjhUMTQzMDAwWiBjaGFybGVzbWluZ0BiZXJrZWxleS5lZHU', 'created': '2023-06-11T23:27:48.000Z', 'updated': '2023-08-28T00:18:34.126Z', 'summary': 'morning routine; weigh; affirmation. read; answer messages', 'description': '<p>teeth/face, weigh, water, dress, hair, pack bag</p>', 'creator': {'email': 'charlesming@berkeley.edu', 'self': True}, 'organizer': {'email': 'charlesming@berkeley.edu', 'self': True}, 'start': {'dateTime': '2023-08-28T07:30:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2023-08-28T08:15:00-07:00', 'timeZone': 'America/Los_Angeles'}, 'recurringEventId': '7je0lph6k52uknelid0an4h8lf_R20230828T143000', 'originalStartTime': {'dateTime': '2023-08-28T07:30:00-07:00', 'timeZone': 

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
# raw_documents = TextLoader('../../../state_of_the_union.txt').load()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# documents = text_splitter.split_documents(raw_documents)
# db = Chroma.from_documents(documents, OpenAIEmbeddings())

# plan: load from json so each outfit is a separate document where the content is the outfit pieces, and the metadata contains all other metadata
# No need to use splitter because chunk size is already good (granularity is per outfit)
# Create embeddings
# Eventually query using weather and event context. Output goes into later chain to generate the full report.

In [8]:
from langchain.prompts.chat import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

human_input = f"""The weather forecast for today is {forecast_data}.
The next ten events in the user's calendar are {events}.
Provide two sentences of insight into the weather. Then, brief me on the day's events. Assume I have a general idea, and want to know if it looks busy, stressful, or any other standouts. 
Given that context, recommend three outfits to me, with a two sentence justification of each one. If possible, suggest times when an outfit change may be needed due to my schedule.
You have creative freedom to give other suggestions that will reduce my decision fatigue for the day."""

human_message_prompt = HumanMessagePromptTemplate(
        prompt = PromptTemplate(
            input_variables=["weather", "events"],
            template="""The weather forecast for today is {weather}.
            The next ten events in the user's calendar are {events}.
            Provide two sentences of insight into the weather. Then brief the user on the day's events. Assume he has a general idea, and wants to know if it looks busy, stressful, or any other standouts. 
            Given that context, recommend three outfits to the user, with a two sentence explanation of each one. If possible, suggest times when an outfit change may be needed due to the user's schedule.
            You have creative freedom to give other suggestions that will reduce my decision fatigue for the day.""",
        )
    )

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=f"You are a flirty assistant for {user}. Include at least one witty joke in each response. Make sure to end the joke with (that was a joke)."), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injected
])
    
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = LLMChain(llm=chat, prompt=prompt, verbose=True, memory=memory)
out = chain.predict(human_input=human_input)
print(out)






> Entering new LLMChain chain...
Prompt after formatting:
System: You are a flirty assistant for {'name': 'Charles', 'age': 21, 'sex': 'male'}. Include at least one witty joke in each response. Make sure to end the joke with (that was a joke).
Human: The weather forecast for today is {'latitude': 37.877266, 'longitude': -122.275635, 'generationtime_ms': 0.7539987564086914, 'utc_offset_seconds': -25200, 'timezone': 'America/Los_Angeles', 'timezone_abbreviation': 'PDT', 'elevation': 56.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°F', 'precipitation_probability': '%', 'precipitation': 'mm'}, 'hourly': {'time': ['2023-08-28T00:00', '2023-08-28T01:00', '2023-08-28T02:00', '2023-08-28T03:00', '2023-08-28T04:00', '2023-08-28T05:00', '2023-08-28T06:00', '2023-08-28T07:00', '2023-08-28T08:00', '2023-08-28T09:00', '2023-08-28T10:00', '2023-08-28T11:00', '2023-08-28T12:00', '2023-08-28T13:00', '2023-08-28T14:00', '2023-08-28T15:00', '2023-08-28T16:00', '2023-08-28T17:00', '2023-08

In [9]:
chain.predict(human_input="Outfit 2 is currently in the laundry. What is an alternative?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a flirty assistant for {'name': 'Charles', 'age': 21, 'sex': 'male'}. Include at least one witty joke in each response. Make sure to end the joke with (that was a joke).
Human: The weather forecast for today is {'latitude': 37.877266, 'longitude': -122.275635, 'generationtime_ms': 0.7539987564086914, 'utc_offset_seconds': -25200, 'timezone': 'America/Los_Angeles', 'timezone_abbreviation': 'PDT', 'elevation': 56.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°F', 'precipitation_probability': '%', 'precipitation': 'mm'}, 'hourly': {'time': ['2023-08-28T00:00', '2023-08-28T01:00', '2023-08-28T02:00', '2023-08-28T03:00', '2023-08-28T04:00', '2023-08-28T05:00', '2023-08-28T06:00', '2023-08-28T07:00', '2023-08-28T08:00', '2023-08-28T09:00', '2023-08-28T10:00', '2023-08-28T11:00', '2023-08-28T12:00', '2023-08-28T13:00', '2023-08-28T14:00', '2023-08-28T15:00', '2023-08-28T16:00', '2023-08-28T17:00', '2023-08

'No worries, I\'ve got you covered! If Outfit 2 is in the laundry, let\'s go with an alternative outfit. How about a pair of khaki pants, a crisp white button-down shirt, and a nice pair of loafers? This combination is classic, versatile, and suitable for the "How to Be a Futurist" event. You\'ll still look polished and put-together without having to worry about the laundry situation. And hey, who knows, maybe white is your lucky color today!\n\nIf you need any more outfit suggestions or have any other requests, feel free to ask. I\'m here to assist you (and keep your wardrobe game strong). 😏'